In [56]:
import base64
import xml.etree.ElementTree as ET
import pandas as pd
import os

In [57]:
def set_bit(v, index, x):
    """
        Set the index:th bit of v to 1 if x is truthy,
        else to 0, and return the new value.
    """
    mask = 1 << index   # Compute mask, an integer with just bit 'index' set.
    v &= ~mask          # Clear the bit indicated by the mask (if x is False)
    if x:
        v |= mask         # If x was True, set the bit indicated by the mask.
    return v 

In [59]:
def decode_xml_file(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()

    data = []

    for cpc in root.findall('cpc'):
        
        seq = cpc.attrib.get('seq', None)
        datetime = cpc.attrib.get('datetime', None)
        tzoffset = cpc.attrib.get('tzoffset', None)
        tz = cpc.attrib.get('tz', None)

        for device in cpc.findall('.//device'):
            for mg in device.findall('.//mg'):
                mg_name = mg.attrib.get('name', None)
                wave = ''
                offset = 0
                gain = 0
                points = 0
                hz = 0
                binwave = []

                for m in mg:                
                    if m.attrib['name'] == 'Offset':
                        offset = int(m.text)
                    elif m.attrib['name'] == 'Gain':
                        if mg_name == 'GE_ART':
                            gain = 0.25
                        elif mg_name == 'INVP1':
                            gain = 0.01
                        else:
                            gain = float(m.text)       

                    elif m.attrib['name'] == 'Hz':
                        hz = int(m.text)   
                    elif m.attrib['name'] == 'Points':
                        points = int(m.text)
                    elif m.attrib['name'] == 'Wave':
                        wave = m.text

                        wave = base64.b64decode(wave)

                for i in range(0, len(wave)-1, 2):
                    t  = (wave[i]) + wave[i+1] * 256
                    t = set_bit(t, 15, 0) + (-32768) * (t >> 15)   

                    t = t * gain + offset

                    binwave.append(t)  

                data.append([seq, datetime, tzoffset, tz, 
                                    mg_name,hz,points,binwave]) 


    df = pd.DataFrame(data, columns=['seq', 'datetime', 'tzoffset', 'tz', 
                                    'mg_name','hz','points','binwave'])                        

    return df            

In [1]:

filePath = r"H:\sml data\epic_wave_2_v2.tar\epic_wave_2_v2\UCI_deidentified_part2_EPIC_08_10\Waveforms\\"
folderName = []
fileName = []